In [ ]:
!pip install pytorch-fid

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
%cd CS839\ Project/ADE20K_outputs

/content/drive/.shortcut-targets-by-id/1GJbVlPKw97sX41_uzJoAvjQmkKWbf0lX/CS839 Project/ADE20K_outputs


In [ ]:
!mkdir -p temp_files/validation_resized
#!mkdir -p temp_files/stylegan_copies

In [ ]:
#!/usr/bin/python
from PIL import Image
import os, sys

path = "validation/"
path_out="temp_files/validation_resized/"
dirs = os.listdir( path )
img_size=(225,225)


for item in dirs:
    if os.path.isfile(path+item):
        im = Image.open(path+item)
        f, e = os.path.splitext(path+item)
        imResize = im.resize(img_size, Image.LANCZOS)
        imResize.save(path_out + item.split(".")[0]+".png" , 'PNG', quality=100)

In [ ]:
def compare_folders(a,b):
  a_n =!ls -la $a|wc -l
  b_n= !ls -la $b|wc -l
  print(a_n[0],b_n[0])

In [ ]:
compare_folders('temp_files/validation_resized','adain')

815 815


In [ ]:
!python -m pytorch_fid temp_files/validation_resized adain_2056

100% 42/42 [00:15<00:00,  2.68it/s]
100% 42/42 [00:15<00:00,  2.76it/s]
FID:  35.52380056973254


In [ ]:
compare_folders('temp_files/validation_resized','temp_files/stylegan_copies')
!python -m pytorch_fid temp_files/validation_resized temp_files/stylegan_copies

total 20
drwx------ 2 root root 4096 Dec  1 00:03 adain/
drwx------ 2 root root 4096 Dec  1 00:03 baseline/
drwx------ 2 root root 4096 Dec  3 20:00 stylegan/
drwx------ 4 root root 4096 Dec  3 21:39 temp_files/
drwx------ 2 root root 4096 Dec  3 20:33 validation/


In [ ]:
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 3.8 MB/s eta 0:00:00


In [ ]:

import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

# create default evaluator for doctests

In [ ]:

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)
default_evaluator2 = Engine(eval_step)
default_evaluator3 = Engine(eval_step)
default_evaluator4 = Engine(eval_step)

In [ ]:
metric = SSIM(data_range=1.0)
metric.attach(default_evaluator, 'ssim')
metric2 = SSIM(data_range=1.0)
metric2.attach(default_evaluator2, 'ssim')
metric3 = SSIM(data_range=1.0)
metric3.attach(default_evaluator3, 'ssim')
metric4 = SSIM(data_range=1.0)
metric4.attach(default_evaluator4, 'ssim')

In [ ]:
transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor()
                               ])

In [ ]:
base_loader=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder('tempbase',transform=transform),batch_size=64,shuffle=False)
val_loader=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder('tempval',transform=transform),batch_size=64,shuffle=False)
adain_loader=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder('tempadain',transform=transform),batch_size=64,shuffle=False)
stylegan_loader=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder('tempstylegan',transform=transform),batch_size=64,shuffle=False)
pretrained_loader=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder('temppretrained',transform=transform),batch_size=64,shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
import tqdm

In [ ]:
base_ssim=[]
adain_ssim=[]
stylegan_ssim=[]
pretrained_ssim=[]

for a,b,c,d,e in tqdm.tqdm(zip(val_loader,base_loader,adain_loader,stylegan_loader,pretrained_loader)):
  state = default_evaluator.run([[b[0], a[0]]])
  state2 = default_evaluator2.run([[c[0], a[0]]])
  state3 = default_evaluator3.run([[d[0], a[0]]])
  state4 = default_evaluator4.run([[e[0], a[0]]])

  base_ssim.append(state.metrics['ssim'])
  adain_ssim.append(state2.metrics['ssim'])
  stylegan_ssim.append(state3.metrics['ssim'])
  pretrained_ssim.append(state4.metrics['ssim'])

13it [37:55, 175.08s/it]


In [ ]:
sum(base_ssim)/len(base_ssim)

0.2941719819360938

In [ ]:
sum(adain_ssim)/len(adain_ssim)

0.2955579455530154

In [ ]:
sum(stylegan_ssim)/len(stylegan_ssim)

0.28926104545545156

In [ ]:
sum(pretrained_ssim)/len(pretrained_ssim)

0.29995734297397586

In [ ]:
!pip install clean-fid

In [ ]:
from cleanfid import fid
score = fid.compute_kid('validation', 'adain')
print(score)

compute KID between two folders


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Found 814 images in the folder validation


KID validation : 100%|██████████| 26/26 [00:32<00:00,  1.27s/it]


Found 814 images in the folder adain


KID adain : 100%|██████████| 26/26 [00:23<00:00,  1.13it/s]


0.009370763952498333


In [ ]:
from cleanfid import fid
score = fid.compute_kid('validation', 'baseline')
print(score)

compute KID between two folders
Found 814 images in the folder validation


KID validation : 100%|██████████| 26/26 [00:33<00:00,  1.30s/it]


Found 814 images in the folder baseline


KID baseline : 100%|██████████| 26/26 [00:24<00:00,  1.07it/s]


0.009807381939157488


In [ ]:
from cleanfid import fid
score = fid.compute_kid('validation', 'stylegan')
print(score)

compute KID between two folders
Found 814 images in the folder validation


KID validation : 100%|██████████| 26/26 [00:27<00:00,  1.05s/it]


Found 814 images in the folder stylegan


KID stylegan : 100%|██████████| 26/26 [00:26<00:00,  1.03s/it]


0.011138581310322096


In [ ]:
from cleanfid import fid
score = fid.compute_kid('validation', 'pretrained')
print(score)

compute KID between two folders
Found 814 images in the folder validation


KID validation : 100%|██████████| 26/26 [00:26<00:00,  1.04s/it]


Found 814 images in the folder pretrained


KID pretrained : 100%|██████████| 26/26 [00:23<00:00,  1.09it/s]


0.0075687405183503835
